In [2]:
# Dependencies
from splinter import Browser
from bs4 import BeautifulSoup
import requests
import pandas as pd
import time

In [3]:
# URL of page to be scraped
url = 'https://mars.nasa.gov/news'

In [4]:
# Retrieve page with the requests module
response = requests.get(url)

In [5]:
# Create BeautifulSoup object; parse with 'lxml'
soup = BeautifulSoup(response.text, 'lxml')

In [6]:
# The article header
news_title = soup.find('div', class_="content_title").text
print(news_title)



NASA Prepares for Moon and Mars With New Addition to Its Deep Space Network




In [7]:
news_synopsis = soup.find('div', class_='rollover_description_inner').text
print(news_synopsis)


Robotic spacecraft will be able to communicate with the dish using radio waves and lasers.



In [8]:
#for row in soup.find_all('div',attrs={"class" : "article_teaser_body"}):
#    print(row)

In [9]:
#Groundwork for splinter
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

In [10]:
pic_url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(pic_url)

In [11]:
html = browser.html
pic_soup = BeautifulSoup(html, 'html.parser')

picture = pic_soup.find('article').find('a').get('data-fancybox-href')

In [12]:
print(picture)

/spaceimages/images/mediumsize/PIA18185_ip.jpg


In [13]:
modified_url = pic_url.split('?')[0]
modified_url

'https://www.jpl.nasa.gov/spaceimages/'

In [14]:
all_content = {}
modified_picture = picture.split('/')[2]+"/"+picture.split('/')[3]+"/"+picture.split('/')[4]
new_url = modified_url + modified_picture
all_content["featured_image"] = new_url

In [15]:
print(all_content["featured_image"])

https://www.jpl.nasa.gov/spaceimages/images/mediumsize/PIA18185_ip.jpg


In [16]:
# Twitter URL of weather to be scraped
twitter_url = 'https://twitter.com/marswxreport?lang=en'
# Retrieve page with the requests module
response = requests.get(twitter_url)
time.sleep(1)
# Create BeautifulSoup object; parse with 'lxml'
twitter_soup = BeautifulSoup(response.text, 'html.parser')
timeline = twitter_soup.select('#timeline li.stream-item')

In [17]:
#print(timeline)

In [18]:
all_tweets = []
for tweet in timeline:
    #tweet_id = timeline['data-item-id']
    tweet_text = tweet.select('p.tweet-text')[0].get_text()
    all_tweets.append({"text": tweet_text})

mars_weather = all_tweets[1]
print(mars_weather)

{'text': 'InSight sol 449 (2020-03-01) low -93.6ºC (-136.4ºF) high -11.3ºC (11.7ºF)\nwinds from the SSW at 6.8 m/s (15.1 mph) gusting to 23.9 m/s (53.4 mph)\npressure at 6.30 hPapic.twitter.com/ZcHyd02Y9r'}


In [19]:
#Mars facts scrape
fact_url = 'https://space-facts.com/mars/'
facts_tables = pd.read_html(fact_url)
facts_tables

[                      0                              1
 0  Equatorial Diameter:                       6,792 km
 1       Polar Diameter:                       6,752 km
 2                 Mass:  6.39 × 10^23 kg (0.11 Earths)
 3                Moons:            2 (Phobos & Deimos)
 4       Orbit Distance:       227,943,824 km (1.38 AU)
 5         Orbit Period:           687 days (1.9 years)
 6  Surface Temperature:                   -87 to -5 °C
 7         First Record:              2nd millennium BC
 8          Recorded By:           Egyptian astronomers,
   Mars - Earth Comparison             Mars            Earth
 0               Diameter:         6,779 km        12,742 km
 1                   Mass:  6.39 × 10^23 kg  5.97 × 10^24 kg
 2                  Moons:                2                1
 3      Distance from Sun:   227,943,824 km   149,598,262 km
 4         Length of Year:   687 Earth days      365.24 days
 5            Temperature:    -153 to 20 °C      -88 to 58°C,
           

In [20]:
facts_df = facts_tables[0]
facts_df.columns = ['Description', 'Value']
facts_df.set_index('Description', inplace=True)
facts_df.head()

,Value
Description,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.39 × 10^23 kg (0.11 Earths)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.38 AU)"


In [21]:
hemisphere_title_clean = []
url_component = []
hemisphere_url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(hemisphere_url)
hemisphere_html = browser.html
hemisphere_soup = BeautifulSoup(hemisphere_html, 'html.parser')
all_hemispheres = hemisphere_soup.find_all('a', class_='itemLink product-item')
hemisphere_title = hemisphere_soup.find_all('h3')
for title in hemisphere_title:
    hemisphere_title_clean.append(title.text.strip())
for hemisphere in all_hemispheres:
    url_component.append(hemisphere['href'])
url_component = list( dict.fromkeys(url_component))
print(hemisphere_url)    

https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars


In [22]:
hemisphere_title_clean
#hemisphere = hemisphere_url.split('/')[2]
#hemisphere

['Cerberus Hemisphere Enhanced',
 'Schiaparelli Hemisphere Enhanced',
 'Syrtis Major Hemisphere Enhanced',
 'Valles Marineris Hemisphere Enhanced']

In [24]:
hemisphere = hemisphere_url.split('/')[2]
image_urls = []
for url in url_component:
    hemisphere_url ='https://astropedia.'+hemisphere+'/download/'+url.split('/')[3]+'/'+url.split('/')[4]+'/'+url.split('/')[5]+'.tif/full.jpg'
    image_urls.append(hemisphere_url)   
        

In [25]:
image_urls

['https://astropedia.astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg',
 'https://astropedia.astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg',
 'https://astropedia.astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg',
 'https://astropedia.astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg']

In [26]:
hemisphere_image_urls = dict(zip(hemisphere_title_clean,image_urls))

In [27]:
hemisphere_image_urls

{'Cerberus Hemisphere Enhanced': 'https://astropedia.astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg',
 'Schiaparelli Hemisphere Enhanced': 'https://astropedia.astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg',
 'Syrtis Major Hemisphere Enhanced': 'https://astropedia.astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg',
 'Valles Marineris Hemisphere Enhanced': 'https://astropedia.astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}